<a href="https://colab.research.google.com/github/KhanhPham2411/DeepLearning_BaNgoc/blob/master/L13/NLP/SentimentFilmReviewClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds

In [2]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePDU1QR/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePDU1QR/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePDU1QR/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [4]:
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

for s, l in train_data:
    train_sentences.append(str(s.numpy()))
    train_labels.append(l.numpy())

for s, l in test_data:
    test_sentences.append(str(s.numpy()))
    test_labels.append(l.numpy())


In [5]:
train_labels[0]

0

In [6]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [7]:
vocab_size = 10000
embedding_dim = 64
max_length = 140

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

In [10]:
def texts_to_padded_sequences(sentences):
    sequences = tokenizer.texts_to_sequences(sentences)
    return pad_sequences(sequences, maxlen=max_length, 
                                       truncating='post', padding="post")

In [11]:
padded_train_sequences = texts_to_padded_sequences(train_sentences)

In [12]:
padded_test_sequences = texts_to_padded_sequences(test_sentences)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [14]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [16]:
model.fit(padded_train_sequences, train_labels, epochs=10, validation_data=(padded_test_sequences, test_labels))

Epoch 1/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4557 - acc: 0.7674 - val_loss: 0.3693 - val_acc: 0.8348
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.1347 - acc: 0.9530 - val_loss: 0.4675 - val_acc: 0.8120
Epoch 3/10
782/782 [==============================] - 12s 15ms/step - loss: 0.0148 - acc: 0.9979 - val_loss: 0.6263 - val_acc: 0.8008
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.6108 - val_acc: 0.8197
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 6.0467e-04 - acc: 1.0000 - val_loss: 0.6502 - val_acc: 0.8203
Epoch 6/10
782/782 [==============================] - 11s 13ms/step - loss: 3.0894e-04 - acc: 1.0000 - val_loss: 0.6812 - val_acc: 0.8211
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 1.7499e-04 - acc: 1.0000 - val_loss: 0.7109 - val_acc: 0.8218
Epoch 8/10
782/782 [==============================

In [17]:
test_seq = texts_to_padded_sequences(["I hate this movie"])

In [18]:
model.predict(test_seq)

array([[0.20180967]], dtype=float32)

In [19]:
import io

def export_embedding_weights(model):
    e = model.layers[0]
    weights = e.get_weights()[0]

    out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
    out_m = io.open('meta.tsv', 'w', encoding='utf-8')

    for idx in range(1, vocab_size):
        word = tokenizer.index_word[idx]
        vector = weights[idx]
        out_m.write(word + "\n")
        out_v.write('\t'.join([str(x) for x in vector]) + "\n")

    out_m.close()
    out_v.close()

In [ ]:
export_embedding_weights(model)